In [ ]:
!pip install pyTelegramBotAPI #устанавливаю библиотеку для телеграм-бота
!pip install bertopic
!pip install emoji

In [ ]:
from bertopic import BERTopic

import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

import emoji
import telebot
from telebot import types
import random

Mounted at /content/drive


In [ ]:
top_df_2 = pd.read_csv('/content/drive/MyDrive/model/top_df_3.csv', index_col=0) # данные, пропущенные через модель
topic_model = BERTopic.load('/content/drive/MyDrive/model/mm3') # модель

In [ ]:
# определение функции возвращения ссылок

def return_links(my_mes):
    my_topic = topic_model.find_topics(my_mes, top_n = 1)[0][0]
    return top_df_2[top_df_2.topic  == my_topic].sample(n=1).link #вывод подходящих ссылок

In [ ]:
botTimeWeb = telebot.TeleBot('6767214855:AAHGuoOnbmlpC5FRpGq6AY-pkbQJxmFDXSI')#здесь уникальный токен бота

#обработчик сообщений message_handler
@botTimeWeb.message_handler(commands=['start'])
def startBot(message):
  first_mess = f"<b>{message.from_user.first_name} {message.from_user.last_name}</b>, привет!\nХочешь я помогу тебе выбрать фильм для просмотра?"+ emoji.emojize(":brain:")
  #создание кнопки типа inline(отображается под сообщением), переправляет пользователя на следующее сообщение
  key = types.InlineKeyboardMarkup()

  button_yes = types.InlineKeyboardButton(text = 'Да', callback_data = 'yes')

  button_no = types.InlineKeyboardButton(text = 'Нет', callback_data = 'no')

  key.add(button_yes, button_no)
  botTimeWeb.send_message(message.chat.id, first_mess, parse_mode='html', reply_markup=key) # бот отправляет сообщение


#функционал кнопки "Да" и "Нет"
@botTimeWeb.callback_query_handler(func=lambda call:call.data == 'yes')
def response(function_call):
        second_mess = "Напиши мне в ответном сообщении, какое у тебя настроение и чего ты ожидаешь от просмотра))"
        botTimeWeb.send_message(function_call.message.chat.id, second_mess)
        # сообщение от пользователя и ответ на него
        @botTimeWeb.message_handler(content_types=["text"])
        def answer_message(message):
            mymes = message.text
            answer = return_links(mymes) #возвращение ссылки на фильм РАНДОМНОЙ!!!

            key = types.InlineKeyboardMarkup()
            button_still = types.InlineKeyboardButton(text = 'Ещё', callback_data = 'still')#тут надо добавить кнопку еще
            key.add(button_still)

            botTimeWeb.send_message(message.chat.id,answer,parse_mode='html', reply_markup=key)

            @botTimeWeb.callback_query_handler(func=lambda call: call.data == 'still')
            def response_(function_call):
               third_mess = return_links(mymes) #возвращение ссылки на фильм
               key = types.InlineKeyboardMarkup()
               button_still = types.InlineKeyboardButton(text = 'Ещё', callback_data = 'still')#тут надо добавить кнопку еще
               key.add(button_still)
               botTimeWeb.send_message(function_call.message.chat.id, third_mess,parse_mode='html', reply_markup=key)
               #обработка запроса кнопки "ещё" закончена
               botTimeWeb.answer_callback_query(function_call.id)


        #обработка запроса кнопки "да" закончена
        botTimeWeb.answer_callback_query(function_call.id)

@botTimeWeb.callback_query_handler(func=lambda call:call.data == 'no')
def response(function_call):
        #if function_call.data == "no": #если нажата кнопка нет
        second_mess = "Что ж...насильно мил не будешь"+ emoji.emojize(":winking_face:")
        botTimeWeb.send_message(function_call.message.chat.id, second_mess)
        #обработка запроса закончена
        botTimeWeb.answer_callback_query(function_call.id)


#непрерывная работа бота
botTimeWeb.infinity_polling()